In [1]:
if False:
    # Install Unsloth, Xformers (Flash Attention), and all other packages
    !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
    !pip install --no-deps "trl<0.9.0" 
    !pip install --no-deps peft accelerate bitsandbytes
    
    # Install Flash Attention 2 for softcapping support
    import torch
    if torch.cuda.get_device_capability()[0] >= 8:
        !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"
    
    !pip install -U triton
    !pip install -U torch==2.2.0
    !pip install "xformers<0.0.26"

In [2]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install git+https://github.com/huggingface/transformers.git
!pip install "xformers<0.0.26"
!pip install peft accelerate bitsandbytes
!pip install "trl<0.9.0"

In [3]:
token = ""
model_id = "google/gemma-2-2b-it"
system_prompt = "You're a helpful assistant that answers math problems."

# 1. Load and Transform Dataset

In [4]:
from datasets import load_dataset

DATASET_NAME = "TIGER-Lab/MathInstruct"
dataset = load_dataset(DATASET_NAME, split='train')

/root/miniconda3/envs/unsloth/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
if False:
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_id, token=token)

    lengths = []
    for idx, item in enumerate(dataset):
        length = len(tokenizer.tokenize(item['instruction'] + '\n' + item['output']))
        lengths.append(length)
        print(idx)
        if idx > 50000:
            break

In [6]:
if False:
    import matplotlib.pyplot as plt

    plt.hist(lengths, range=[0,1000], bins=50, density=True, cumulative=True)
    plt.show()

In [7]:
# Remove duplicates

from collections import Counter

instruction_counts = Counter(dataset['instruction'])
duplicates = {instruction: count for instruction, count in instruction_counts.items() if count > 1}
def filter_unique_examples(example):
    return instruction_counts[example['instruction']] == 1

filtered_dataset = dataset.filter(filter_unique_examples)

In [8]:
len(filtered_dataset), len(dataset)

(208826, 262039)

In [9]:
# Transform data
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, token=token)

def formatting_prompt(item):
    chat = [
        { "role": "user", "content": system_prompt + '\n' + item['instruction'] },
        { "role": "model", "content": item['output'] }
    ]
    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=False)
    return {'text': prompt}
    
formatted_dataset = filtered_dataset.map(formatting_prompt)

In [10]:
formatted_dataset[0]

{'source': 'data/CoT/aqua_rat.json',
 'output': "Let's think about the multi-choice question.\n6.52 × 10^5 ly / (3.26 ly/parsec) = 2 x 10^5 persec\nThe answer is A.",
 'instruction': 'The distance between two stars is 6.52 × 10^5 light years. What is the distance between the two stars in parsecs? (1 parsec = 3.26 light years)\nAnswer Choices: (A) 2 × 10^5 (B) 4 × 10^6 (C) 5 × 10^7 (D) 7 × 10^7 (E) 9 × 10^8',
 'text': "<bos><start_of_turn>user\nYou're a helpful assistant that answers math problems.\nThe distance between two stars is 6.52 × 10^5 light years. What is the distance between the two stars in parsecs? (1 parsec = 3.26 light years)\nAnswer Choices: (A) 2 × 10^5 (B) 4 × 10^6 (C) 5 × 10^7 (D) 7 × 10^7 (E) 9 × 10^8<end_of_turn>\n<start_of_turn>model\nLet's think about the multi-choice question.\n6.52 × 10^5 ly / (3.26 ly/parsec) = 2 x 10^5 persec\nThe answer is A.<end_of_turn>\n"}

In [11]:
split_dataset = formatted_dataset.train_test_split(test_size=0.005, seed=3407)

ds_train = split_dataset['train']
ds_valid = split_dataset['test']

In [12]:
ds_train

Dataset({
    features: ['source', 'output', 'instruction', 'text'],
    num_rows: 207781
})

# 2. Load Model

In [13]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 512 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# base_model_id = "unsloth/gemma-2-2b-it"
base_model_id = "unsloth/gemma-2-2b-it-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = base_model_id,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = token
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: If you want to finetune Gemma 2, install flash-attn to make it faster!
To install flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
==((====))==  Unsloth 2024.8: Fast Gemma2 patching. Transformers = 4.45.0.dev0.
   \\   /|    GPU: NVIDIA GeForce RTX 3080 Ti. Max memory: 11.73 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.25.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [14]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0.05, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.8 patched 26 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


# 3. Finetuning

In [15]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = ds_train,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.01,
        num_train_epochs = 3, # Set this for 1 full training run.
#         max_steps = 60,
        learning_rate = 1e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "paged_adamw_8bit",
        weight_decay = 0.01,
        # lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        save_strategy = 'epoch'
    ),
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [16]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 112,979 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 4
\        /    Total batch size = 32 | Total steps = 10,590
 "-____-"     Number of trainable parameters = 83,066,880
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: shubhamdawande-sd. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,2.135200
2,2.057400
3,2.337000
4,2.187600
5,2.110100
6,2.105700
7,2.041400
8,2.087300
9,2.142700
10,2.066600


In [17]:
if True:
    model.save_pretrained("gemma2_lora")
    tokenizer.save_pretrained("gemma2_lora")

if False:
    # model.save_pretrained_merged("gemma2_16bit", tokenizer, save_method = "merged_16bit",)
    model.push_to_hub_merged("morpheus73/gemma2_16bit", tokenizer, save_method = "merged_16bit", token = token)

In [18]:
# print(ds_valid[0]['text'])

In [19]:
# sample = """<bos><start_of_turn>user\nan object thrown directly upward is at a height of h feet after t seconds , where h = - 12 ( t - 3 ) ^ 2 + 120 . at what height , in feet , is the object 2 seconds after it reaches its maximum height ? Let's write a program.<end_of_turn>\n<start_of_turn>model
# """

In [20]:
# FastLanguageModel.for_inference(model) # Enable native 2x faster inference
# inputs = tokenizer([sample], return_tensors = "pt").to("cuda")

# outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
# print(tokenizer.batch_decode(outputs)[0])